## Loading libraries 

In [73]:
library(dplyr)
library(tidyr)
library(tibble)
library(lubridate)
library(readr)
library(stringr)
library(ggplot2)
library(data.table)
library(odbc)
library(RMariaDB)

## DB Connection 

In [74]:
con <- dbConnect(
  drv = RMariaDB::MariaDB(),
  username = "andrea.perezv",
  password = "Aeph9eF3",
  host = "ehr3.deim.urv.cat",
  dbname = "mimiciiiv14",
  port = 3306
)

In [75]:
#con2 <- dbConnect(
 # drv = RMariaDB::MariaDB(),
  #username = "andrea.perezv",
  #password = "Aeph9eF3",
  #host = "ehr3.deim.urv.cat",
  #dbname = "mimiciv_hosp",
  #port = 3306
#)

## Overview

In [76]:
dbListTables(con)

[1] "DATETIMEEVENTS"     "MICROBIOLOGYEVENTS" "D_CPT"             
 [4] "ICUSTAYS"           "PROCEDURES_ICD"     "CHARTEVENTS"       
 [7] "SERVICES"           "CALLOUT"            "NOTEEVENTS"        
[10] "D_ITEMS"            "ADMISSIONS"         "PRESCRIPTIONS"     
[13] "INPUTEVENTS_CV"     "INPUTEVENTS_MV"     "PATIENTS"          
[16] "DRGCODES"           "D_LABITEMS"         "OUTPUTEVENTS"      
[19] "DIAGNOSES_ICD"      "PROCEDUREEVENTS_MV" "LABEVENTS"         
[22] "CAREGIVERS"         "D_ICD_DIAGNOSES"    "CPTEVENTS"         
[25] "TRANSFERS"          "D_ICD_PROCEDURES"

In [77]:
dbGetQuery(con, "SHOW DATABASES;")


Database
<chr>
information_schema
mimiciiiv14


We do not have connection to other DBs from MIMIC like mimic_icu or mimic_derived. We will compute the most part of variables by ourselves. 

## Samples selection 
For each patient (subject_id), find the earliest ICU admission time (MIN(intime)).

After this runs, df_icu will have columns:

- subject_id → patient ID

- hadm_id → hospital admission ID

- icustay_id → ICU stay ID

- intime → ICU admission time

- outtime → ICU discharge time

In [78]:
sql_first_icu <- "
SELECT
    icu.subject_id,
    icu.hadm_id,
    icu.icustay_id,
    icu.intime,
    icu.outtime
FROM ICUSTAYS icu
JOIN (
    SELECT subject_id, MIN(intime) AS first_intime
    FROM ICUSTAYS
    GROUP BY subject_id
) first_icu
  ON icu.subject_id = first_icu.subject_id
 AND icu.intime = first_icu.first_intime
;
"

df_icu <- dbGetQuery(con, sql_first_icu)

### Add sociodemographic information 

In [79]:
sql_adm_pat <- "
SELECT 
    a.hadm_id,
    a.admittime AS hosp_admit,
    a.dischtime AS hosp_disch,
    p.gender,
    a.ethnicity,
    p.dob,
    p.dod
FROM ADMISSIONS a
JOIN PATIENTS p ON a.subject_id = p.subject_id;
"

df_adm_pat <- dbGetQuery(con, sql_adm_pat)

### Merge ICU with admissions + patient info

In [80]:
df_icu <- df_icu %>%
  inner_join(df_adm_pat, by = "hadm_id") %>%
  
  # Compute ICU and hospital LOS in hours
  mutate(
    icu_los_hours = as.numeric(difftime(outtime, intime, units = "hours")),
    hosp_los_hours = as.numeric(difftime(hosp_disch, hosp_admit, units = "hours"))
  ) %>%
  
  # Keep only ICU stays >= 24h
  filter(icu_los_hours >= 24) %>%
  
  # Clean gender and ethnicity
  filter(
    !is.na(gender), gender != "UNKNOWN",
    !is.na(ethnicity), ethnicity != "UNKNOWN/NOT SPECIFIED"
  ) %>%
  
  # Group ethnicity into major categories
  mutate(
    ethnicity_group = case_when(
      grepl("^WHITE", ethnicity) ~ "WHITE",
      grepl("^BLACK", ethnicity) ~ "BLACK",
      grepl("^HISPANIC", ethnicity) ~ "HISPANIC",
      grepl("^ASIAN", ethnicity) ~ "ASIAN",
      TRUE ~ "OTHER"
    ),
    ethnicity_group = factor(ethnicity_group, levels = c("WHITE", "BLACK", "HISPANIC", "ASIAN", "OTHER"))
  )

### Filtering by age and other missing information 

In [81]:
df_icu <- df_icu %>%
  # Calculate age at ICU admission
  mutate(AGE = as.numeric(format(intime, "%Y")) - as.numeric(format(dob, "%Y"))) %>%
  
  # Keep only adult patients between 18 and 85 years
  filter(AGE >= 18 & AGE <= 85) %>%
  
  # Define mortality: 1 if patient died before or during hospital discharge
  mutate(MORTALITY = ifelse(!is.na(dod) & dod <= hosp_disch, 1, 0))

## Adding Comorbidity Flags
**Binary comorbidity flags** to the existing cleaned ICU cohort. 
The flags are derived from ICD-9 diagnosis descriptions.

In [82]:
dx <- dbGetQuery(con, "SELECT * FROM DIAGNOSES_ICD")       # Diagnoses table
d_icd_diagnoses <- dbGetQuery(con, "SELECT * FROM D_ICD_DIAGNOSES")  # ICD definitions

# Keep only diagnoses for our cohort
df_diag <- dx %>%
  filter(HADM_ID %in% df_icu$hadm_id) %>%
  left_join(d_icd_diagnoses, by = c("ICD9_CODE" = "ICD9_CODE"))

# Create comorbidity flags
df_flags <- df_diag %>%
  group_by(HADM_ID) %>%
  summarise(
    flag_diabetes     = max(grepl("diabetes", tolower(LONG_TITLE))),
    flag_hypertension = max(grepl("hypertension|high blood pressure", tolower(LONG_TITLE))),
    flag_ckd          = max(grepl("chronic kidney|renal failure|kidney failure", tolower(LONG_TITLE))),
    flag_chf          = max(grepl("heart failure|congestive heart", tolower(LONG_TITLE))),
    flag_copd         = max(grepl("copd|chronic obstructive|emphysema|chronic bronchitis", tolower(LONG_TITLE))),
    flag_cancer       = max(grepl("malignan|cancer|carcinoma|neoplasm|tumor", tolower(LONG_TITLE)))
  )%>%
    rename(hadm_id = HADM_ID)

# Merge comorbidity flags with ICU dataset
df_icu <- df_icu %>%
  left_join(df_flags, by = "hadm_id")

##  Pull diagnosis group information from ADMISSIONS


In [83]:
diagnosis_group <- dbGetQuery(con, "
SELECT
  HADM_ID,
  CASE
    -- Cardiovascular
    WHEN LOWER(DIAGNOSIS) LIKE '%coronary%'
      OR LOWER(DIAGNOSIS) LIKE '%myocard%'
      OR LOWER(DIAGNOSIS) LIKE '%angina%'
      OR LOWER(DIAGNOSIS) LIKE '%cardiac%'
      OR LOWER(DIAGNOSIS) LIKE '%chf%'
      OR LOWER(DIAGNOSIS) LIKE '%valve%'
      OR LOWER(DIAGNOSIS) LIKE '%tachy%'
      OR LOWER(DIAGNOSIS) LIKE '%brady%' 
      THEN 'cardiovascular'

    -- Neurological
    WHEN LOWER(DIAGNOSIS) LIKE '%cva%'
      OR LOWER(DIAGNOSIS) LIKE '%seiz%'
      OR LOWER(DIAGNOSIS) LIKE '%brain%'
      OR LOWER(DIAGNOSIS) LIKE '%head%'
      OR LOWER(DIAGNOSIS) LIKE '%spinal%'
      OR LOWER(DIAGNOSIS) LIKE '%cereb%'
      THEN 'neurological'

    -- Infectious
    WHEN LOWER(DIAGNOSIS) LIKE '%sepsis%'
      OR LOWER(DIAGNOSIS) LIKE '%pneumon%'
      OR LOWER(DIAGNOSIS) LIKE '%infection%'
      THEN 'infectious'

    -- Renal
    WHEN LOWER(DIAGNOSIS) LIKE '%renal%'
      OR LOWER(DIAGNOSIS) LIKE '%arf%'
      THEN 'renal'

    -- Respiratory
    WHEN LOWER(DIAGNOSIS) LIKE '%respir%'
      OR LOWER(DIAGNOSIS) LIKE '%copd%'
      OR LOWER(DIAGNOSIS) LIKE '%tracheal%'
      OR LOWER(DIAGNOSIS) LIKE '%pulmonary%'
      THEN 'respiratory'

    -- Oncology
    WHEN LOWER(DIAGNOSIS) LIKE '%cancer%'
      OR LOWER(DIAGNOSIS) LIKE '%lymphoma%'
      OR LOWER(DIAGNOSIS) LIKE '%tumor%'
      OR LOWER(DIAGNOSIS) LIKE '%hemangioma%'
      THEN 'oncology'

    -- Trauma / Musculoskeletal
    WHEN LOWER(DIAGNOSIS) LIKE '%fracture%'
      OR LOWER(DIAGNOSIS) LIKE '%hernia%'
      OR LOWER(DIAGNOSIS) LIKE '%spinal%'
      OR LOWER(DIAGNOSIS) LIKE '%post-operative%'
      OR LOWER(DIAGNOSIS) LIKE '%sda%'
      THEN 'trauma'

    -- Gastrohepatic / Liver / Biliary
    WHEN LOWER(DIAGNOSIS) LIKE '%liver%'
      OR LOWER(DIAGNOSIS) LIKE '%cirrhosis%'
      OR LOWER(DIAGNOSIS) LIKE '%biliary%'
      OR LOWER(DIAGNOSIS) LIKE '%pancreatic%'
      OR LOWER(DIAGNOSIS) LIKE '%chole%'
      THEN 'gastrohepatic'

    -- Metabolic / Endocrine
    WHEN LOWER(DIAGNOSIS) LIKE '%hypoglycem%'
      OR LOWER(DIAGNOSIS) LIKE '%hyperglycem%'
      OR LOWER(DIAGNOSIS) LIKE '%dehydration%'
      OR LOWER(DIAGNOSIS) LIKE '%failure to thrive%'
      THEN 'metabolic'

    -- Psychiatric / Toxicology
    WHEN LOWER(DIAGNOSIS) LIKE '%overdose%'
      OR LOWER(DIAGNOSIS) LIKE '%withdrawal%'
      OR LOWER(DIAGNOSIS) LIKE '%assault%'
      THEN 'psychiatric'

    -- Neonatal
    WHEN LOWER(DIAGNOSIS) LIKE '%newborn%'
      THEN 'neonatal'

    -- Hematologic / Other
    WHEN LOWER(DIAGNOSIS) LIKE '%splenomegaly%'
      OR LOWER(DIAGNOSIS) LIKE '%anemia%'
      THEN 'hematologic'

    -- Catch-all (everything else)
    ELSE 'other'
  END AS diagnosis_group
FROM ADMISSIONS
")%>%
rename(hadm_id = HADM_ID)


### Check distribution of diagnosis groups

In [84]:
dim(df_icu)

[1] 26504    22

In [85]:
table(diagnosis_group$diagnosis_group)


cardiovascular  gastrohepatic    hematologic     infectious      metabolic 
          8294            938            276           4508            488 
      neonatal   neurological       oncology          other    psychiatric 
          7823           2163            821          26245            654 
         renal    respiratory         trauma 
          1095           1893           3778 

### Merge diagnosis_group into df_icu using HADM_ID
 Merge must be done using HADM_ID (hospital admission ID), 
 because merging by SUBJECT_ID would duplicate rows for each ICU stay a patient has.

In [86]:
df_icu <- df_icu %>%
  left_join(diagnosis_group, by = "hadm_id")

In [87]:
table(df_icu$diagnosis_group)
str(df_icu)


cardiovascular  gastrohepatic    hematologic     infectious      metabolic 
          5255            540            111           1820            208 
  neurological       oncology          other    psychiatric          renal 
          1144            481          13126            376            554 
   respiratory         trauma 
           875           2014 

'data.frame':	26504 obs. of  23 variables:
 $ subject_id       : int  3 4 6 11 12 13 17 18 20 22 ...
 $ hadm_id          : int  145834 185777 107064 194540 112213 143045 194023 188822 157681 165315 ...
 $ icustay_id       : int  211552 294638 228232 229441 232669 263738 277042 298129 264490 204798 ...
 $ intime           : POSIXct, format: "2101-10-20 19:10:11" "2191-03-16 00:29:31" ...
 $ outtime          : POSIXct, format: "2101-10-26 20:43:09" "2191-03-17 16:46:31" ...
 $ hosp_admit       : POSIXct, format: "2101-10-20 19:08:00" "2191-03-16 00:28:00" ...
 $ hosp_disch       : POSIXct, format: "2101-10-31 13:58:00" "2191-03-23 18:41:00" ...
 $ gender           : chr  "M" "F" "F" "F" ...
 $ ethnicity        : chr  "WHITE" "WHITE" "WHITE" "WHITE" ...
 $ dob              : POSIXct, format: "2025-04-11" "2143-05-12" ...
 $ dod              : POSIXct, format: "2102-06-14" NA ...
 $ icu_los_hours    : num  145.5 40.3 88.2 38 183.2 ...
 $ hosp_los_hours   : num  259 186 393 613 305 ...
 $ e

## Limit the temporal window -> last 100 years
 We want to restrict the study period to the most recent 100 years.
 It wouldn’t make sense to analyze data spanning more than a century.
 (Since this may be a simulation, we could adjust this, 
  but here we stick to 100 years.)

In [88]:
# Find the maximum discharge date in the dataset
max_adm <- max(df_icu$hosp_disch, na.rm = TRUE)

# Calculate cutoff date 100 years before the latest discharge
# POSIXct stores time in seconds, so multiply years by days, hours, minutes, and seconds
cutoff <- max_adm - (100 * 365.25 * 24 * 60 * 60)

# Keep only ICU stays with hospital admission after the cutoff
df_icu <- df_icu %>%
  filter(hosp_admit >= cutoff)

 The change in the number of samples was not drastic, 
 but now our study has more clinical relevance by focusing on the most recent 100 years.
 In a real study, the temporal window would likely be much smaller, 
 since disease patterns change over time and societal factors evolve. 
 For example, we cannot directly compare disease incidence in 1800 with that in 2026. 
 Even looking at data from 2000 onwards would introduce strong temporal biases, 
 not to mention the impact of events like the COVID-19 pandemic.

## BMI and BMI changes

### Height 
 We extract the most reliable height measurement for each ICU stay.
 There are two item IDs for height:
 - 226730: height in cm, convert to meters (/100)
 - 226707: height in inches, convert to meters (*0.0254)
 We take the MAX in case multiple entries exist per ICU stay.

In [89]:
sql_height <- "
SELECT
    subject_id,
    icustay_id,
    MAX(
      CASE
        WHEN itemid = 226730 THEN valuenum / 100
        WHEN itemid = 226707 THEN valuenum * 0.0254
      END
    ) AS height_m
FROM CHARTEVENTS
WHERE itemid IN (226730, 226707)
  AND valuenum IS NOT NULL
GROUP BY subject_id, icustay_id;
"

#Execute the SQL and store as a data frame
df_height <- DBI::dbGetQuery(con, sql_height)

# Preview the result
head(df_height)


,subject_id,icustay_id,height_m
,<int>,<int>,<dbl>
1,34,290505,1.6510
2,36,241249,1.8034
3,107,264253,1.6800
4,109,222630,1.5240
5,109,232807,1.5240
6,109,243978,1.5000


In [90]:
# Merge height information into df_icu by icustay_id to ensure each ICU stay gets the correct height
df_icu <- df_icu %>%
  left_join(df_height, by = c("icustay_id", "subject_id"))

# Verify the merge
summary(df_icu$height_m)


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
  0.000   1.630   1.702   1.698   1.780   4.450   18093 

In [91]:
summary(df_height)

   subject_id      icustay_id        height_m    
 Min.   :   34   Min.   :200001   Min.   :0.000  
 1st Qu.:46491   1st Qu.:225267   1st Qu.:1.630  
 Median :63758   Median :250435   Median :1.702  
 Mean   :62008   Mean   :250244   Mean   :1.689  
 3rd Qu.:81676   3rd Qu.:275359   3rd Qu.:1.780  
 Max.   :99999   Max.   :299957   Max.   :4.450  
                 NA's   :3                       

### Remove unrealistic height values to ensure valid BMI calculation
Heights below 1.5 m or above 1.90 m are considered outliers or likely measurement errors.
These rows will be excluded to compute meaningful BMI values.


In [92]:
df_icu <- df_icu %>%
  filter(height_m >= 1.5, height_m <= 1.90)
summary(df_icu$height_m)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.500   1.630   1.702   1.701   1.780   1.880 

## Get baseline weight for each ICU stay
 Two item IDs exist for weight:
 - 226512: weight in pounds, convert to kg (*0.453592)
 - 226531: weight in kg, keep as is
 We take the average of all measurements within the first 24 hours of ICU admission
 to get a baseline weight.

In [93]:
sql_weight_baseline <- "
SELECT
    icu.icustay_id,
    AVG(
      CASE
        WHEN ce.itemid = 226512 THEN ce.valuenum
        WHEN ce.itemid = 226531 THEN ce.valuenum * 0.453592
      END
    ) AS weight_baseline_kg
FROM ICUSTAYS icu
JOIN CHARTEVENTS ce
  ON icu.icustay_id = ce.icustay_id
WHERE ce.itemid IN (226512, 226531)
  AND ce.valuenum IS NOT NULL
  AND ce.charttime BETWEEN icu.intime
                        AND DATE_ADD(icu.intime, INTERVAL 24 HOUR)
GROUP BY icu.icustay_id
;
"

df_weight_baseline <- DBI::dbGetQuery(con, sql_weight_baseline)
head(df_weight_baseline)


,icustay_id,weight_baseline_kg
,<int>,<dbl>
1,200001,60.93602
2,200010,49.25737
3,200011,101.29819
4,200016,64.00000
5,200021,82.50883
6,200028,83.88253


In [94]:
df_icu <- df_icu %>%
  left_join(df_weight_baseline, by = "icustay_id")
summary(df_icu$weight_baseline_kg)


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
  23.86   68.16   80.83   83.61   95.09  370.35     378 

### Remove weight outliers 

In [95]:
df_icu <- df_icu %>%
  filter(weight_baseline_kg >= 35, weight_baseline_kg <= 120)
summary(df_icu$weight_baseline_kg)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  35.24   67.59   79.54   79.99   91.81  120.00 

We though about include the discharge body weigh but we wanted only the baseline information because is the relevance of the attempt of predicting and clustering the mortality on different ICU profiles.

### BMI Calculation 

In [96]:
df_icu <- df_icu %>%
 mutate(
    bmi_baseline  = weight_baseline_kg  / (height_m ^ 2))
# Filtering of implausible values
df_icu <- df_icu %>%
  filter(
    bmi_baseline  > 10 & bmi_baseline  < 80
  )


In [97]:
summary(df_icu$bmi_baseline)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  11.32   23.94   27.22   27.70   30.87   49.89 

## Respiratory SOFA score per ICU stay.

In [98]:
# Look for FiO2 ITEMIDs
sql_fio2_items <- "
SELECT itemid, label, category, unitname
FROM D_ITEMS
WHERE LOWER(label) LIKE '%fio2%'
   OR LOWER(label) LIKE '%fraction of inspired oxygen%'
;"
df_fio2_items <- dbGetQuery(con, sql_fio2_items)

head(df_fio2_items)


,itemid,label,category,unitname
,<int>,<chr>,<chr>,<chr>
1,185,FIO2 Alarm-High,NA,NA
2,186,FIO2 Alarm-Low,NA,NA
3,189,FiO2 (Analyzed),NA,NA
4,190,FiO2 Set,NA,NA
5,191,FiO2/O2 Delivered,NA,NA
6,727,Vision FiO2,NA,NA


In [99]:
# Look for PaO2 ITEMIDs

sql_pao2_items <- "
SELECT itemid, label
FROM D_ITEMS
WHERE LOWER(label) LIKE '%pao2%'
   OR LOWER(label) LIKE '%arterial oxygen%'
;"
df_pao2_items <- dbGetQuery(con, sql_pao2_items)
head(df_pao2_items)


,itemid,label
,<int>,<chr>
1,490,PAO2
2,779,Arterial PaO2


In [101]:
# FiO2 per ICU stay (first 24h)
sql_fio2 <- "
SELECT
    ce.icustay_id,
    MIN(valuenum) AS fio2
FROM CHARTEVENTS ce
JOIN ICUSTAYS icu ON ce.icustay_id = icu.icustay_id
WHERE ce.itemid IN (189, 190, 191, 727)  
  AND ce.charttime BETWEEN icu.intime AND DATE_ADD(icu.intime, INTERVAL 24 HOUR)
GROUP BY ce.icustay_id;
"

df_fio2 <- dbGetQuery(con, sql_fio2 )


In [102]:
# PaO2 per ICU stay (first 24h)

sql_pao2 <- "
SELECT
    icu.icustay_id,
    MIN(ce.valuenum) AS pao2_mmHg
FROM ICUSTAYS icu
JOIN CHARTEVENTS ce
  ON icu.icustay_id = ce.icustay_id
WHERE ce.itemid IN (490, 779)
  AND ce.charttime BETWEEN icu.intime
                        AND DATE_ADD(icu.intime, INTERVAL 24 HOUR)
  AND ce.valuenum IS NOT NULL
GROUP BY icu.icustay_id
;
"

df_pao2 <- dbGetQuery(con, sql_pao2)

In [103]:
# Compute PF ratio & Respiratory SOFA points
df_resp <- df_pao2 %>%
  left_join(df_fio2, by="icustay_id") %>%
  mutate(
    pf_ratio = pao2_mmHg / fio2,  # FiO2 as fraction if stored in % (0–100)
    resp_score = case_when(
      pf_ratio > 400 ~ 0,
      pf_ratio > 300 ~ 1,
      pf_ratio > 200 ~ 2,
      pf_ratio > 100 ~ 3,
      TRUE ~ 4
    )
  )

In [104]:
# Merge info 
df_icu <- df_icu %>%
  left_join(
    df_resp %>% select(icustay_id, pf_ratio, resp_score), 
    by = "icustay_id"
  )

## Cardiovascular SOFA score per ICU stay

SOFA CV scoring is based on:

Score	MAP / Vasopressors
0	MAP ≥ 70 mmHg, no vasopressors
1	MAP < 70 mmHg, no vasopressors
2	Dopamine ≤5 µg/kg/min or dobutamine (any dose)
3	Dopamine 5.1–15 µg/kg/min, epinephrine ≤0.1 µg/kg/min, norepinephrine ≤0.1 µg/kg/min
4	Dopamine >15 µg/kg/min, epinephrine >0.1 µg/kg/min, norepinephrine >0.1 µg/kg/min

In [105]:
# Mean Arterial Pression 
sql_map <- "
SELECT icu.icustay_id,
       MIN(ce.valuenum) AS map_min
FROM ICUSTAYS icu
JOIN CHARTEVENTS ce ON icu.icustay_id = ce.icustay_id
WHERE ce.itemid = 220051  -- MAP ITEMID
  AND ce.charttime BETWEEN icu.intime AND DATE_ADD(icu.intime, INTERVAL 24 HOUR)
  AND ce.valuenum IS NOT NULL
GROUP BY icu.icustay_id;
"

df_cv <- dbGetQuery(con, sql_map)

In [106]:
# Find vasopressor ITEMIDs
sql_vaso_items <- "
SELECT itemid, label
FROM D_ITEMS
WHERE LOWER(label) LIKE '%dopamine%'
   OR LOWER(label) LIKE '%dobutamine%'
   OR LOWER(label) LIKE '%epinephrine%'
   OR LOWER(label) LIKE '%norepinephrine%';
"

df_vaso_items <- dbGetQuery(con, sql_vaso_items)
df_vaso_items

itemid,label
<int>,<chr>
3112,epinephrine mcg/min
4501,DOPAMINE DRIP
5747,dobutamine
5805,DOPAMINE MICS/KG/MIN
5329,Dopamine
30043,Dopamine
30119,Epinephrine-k
30306,Dobutamine Drip
30307,Dopamine Drip


In [107]:
library(dplyr)
library(tidyr)
library(DBI)

# 1️⃣ Vasopressor IDs
vaso_ids <- c(3112,4501,5747,5805,5329,30043,30119,30306,30307,30309,30042,30044,221289,221653,221662,221906)
vaso_cols <- paste0("X", vaso_ids)  # add "X" prefix for valid R column names

# 2️⃣ Pull MAP (min in first 24h)
sql_map <- "
SELECT icu.icustay_id,
       MIN(ce.valuenum) AS map_min
FROM ICUSTAYS icu
JOIN CHARTEVENTS ce
  ON icu.icustay_id = ce.icustay_id
WHERE ce.itemid = 220051
  AND ce.valuenum IS NOT NULL
  AND ce.charttime BETWEEN icu.intime AND DATE_ADD(icu.intime, INTERVAL 24 HOUR)
GROUP BY icu.icustay_id;
"
df_cv <- dbGetQuery(con, sql_map)

# 3️⃣ Pull vasopressors (max rate in first 24h)
sql_vaso <- sprintf("
SELECT im.icustay_id,
       im.itemid,
       MAX(im.rate) AS max_rate_24h
FROM INPUTEVENTS_MV im
JOIN ICUSTAYS icu ON im.icustay_id = icu.icustay_id
WHERE im.itemid IN (%s)
  AND im.starttime BETWEEN icu.intime AND DATE_ADD(icu.intime, INTERVAL 24 HOUR)
GROUP BY im.icustay_id, im.itemid;
", paste(vaso_ids, collapse = ","))

df_vaso <- dbGetQuery(con, sql_vaso)

# 4️⃣ Pivot wide and fix column names
df_vaso_wide <- df_vaso %>%
  mutate(itemid = paste0("X", itemid)) %>%  # prefix X for numeric columns
  pivot_wider(names_from = itemid, values_from = max_rate_24h)

# Update vaso_cols to match pivoted dataframe
vaso_cols <- intersect(vaso_cols, colnames(df_vaso_wide))

# 5️⃣ Combine MAP + vasopressors & compute CV score
df_cv_full <- df_cv %>%
  left_join(df_vaso_wide, by = "icustay_id") %>%
  mutate(
    # Any vasopressor administered
    any_vaso = rowSums(!is.na(across(all_of(vaso_cols)))) > 0,
    
    # Compute CV score using available columns only
    cv_score = case_when(
      # MAP >= 70 & no vasopressors
      map_min >= 70 & !any_vaso ~ 0,
      
      # MAP < 70 & no vasopressors
      map_min < 70 & !any_vaso ~ 1,
      
      # Low-dose or any vasopressor (adjust thresholds if needed)
      !is.na(X221653) | !is.na(X221289) ~ 2,
      
      # Moderate-dose: X221662 5–15 OR X221906 ≤0.1 (example thresholds)
      (!is.na(X221662) & X221662 > 5 & X221662 <= 15) |
      (!is.na(X221906) & X221906 <= 0.1) ~ 3,
      
      # High-dose: X221662 > 15 OR X221906 > 0.1
      (!is.na(X221662) & X221662 > 15) |
      (!is.na(X221906) & X221906 > 0.1) ~ 4,
      
      TRUE ~ NA_real_  # fallback
    )
  )


In [108]:
#df_cv_joined <- df_cv %>% left_join(df_vaso_wide, by = "icustay_id")
#colnames(df_cv_joined)

In [109]:
colnames(df_cv_full)


[1] "icustay_id" "map_min"    "X221662"    "X221906"    "X221289"   
[6] "X221653"    "any_vaso"   "cv_score"

In [110]:
# Merge info 
df_icu <- df_icu %>%
  left_join(
    df_cv_full %>% select(icustay_id, map_min, any_vaso, cv_score), 
    by = "icustay_id"
  )

## Liver SOFA score per ICU stay (Bilirubin)

ITEMIDs (LABEVENTS):

50885 → Bilirubin, total, mg/dL

50882 → Bilirubin, total

In [111]:
sql_liver <- "
SELECT icu.icustay_id,
       MAX(le.valuenum) AS bilirubin
FROM ICUSTAYS icu
JOIN LABEVENTS le ON icu.hadm_id = le.hadm_id
WHERE le.itemid IN (50885, 50882)
  AND le.valuenum IS NOT NULL
  AND le.charttime BETWEEN icu.intime AND DATE_ADD(icu.intime, INTERVAL 24 HOUR)
GROUP BY icu.icustay_id;
"

df_liver <- dbGetQuery(con, sql_liver)

df_liver <- df_liver %>%
  mutate(liver_score = case_when(
    bilirubin < 1.2 ~ 0,
    bilirubin < 2   ~ 1,
    bilirubin < 6   ~ 2,
    bilirubin < 12  ~ 3,
    TRUE            ~ 4
  ))

In [112]:
head(df_liver)

,icustay_id,bilirubin,liver_score
,<int>,<dbl>,<dbl>
1,200001,28,4
2,200003,25,4
3,200006,31,4
4,200007,24,4
5,200009,24,4
6,200010,26,4


In [113]:
# Merge info 
df_icu <- df_icu %>%
  left_join(
    df_liver %>% select(icustay_id, bilirubin, liver_score), 
    by = "icustay_id"
  )

## CNS (GCS) SOFA score per ICU stay

ITEMIDs (CHARTEVENTS):

198 → GCS Eye

199 → GCS Verbal

200 → GCS Motor

201, 202 → total GCS sometimes recorded

In [114]:
sql_gcs <- "
SELECT icu.icustay_id,
       MIN(ce.valuenum) AS gcs
FROM ICUSTAYS icu
JOIN CHARTEVENTS ce ON icu.icustay_id = ce.icustay_id
WHERE ce.itemid IN (198,199,200,201,202)
  AND ce.valuenum IS NOT NULL
  AND ce.charttime BETWEEN icu.intime AND DATE_ADD(icu.intime, INTERVAL 24 HOUR)
GROUP BY icu.icustay_id;
"

df_cns <- dbGetQuery(con, sql_gcs)

df_cns <- df_cns %>%
  mutate(cns_score = case_when(
    gcs == 15 ~ 0,
    gcs >= 13 ~ 1,
    gcs >= 10 ~ 2,
    gcs >= 6  ~ 3,
    TRUE      ~ 4
  ))

In [115]:
head(df_cns)

,icustay_id,gcs,cns_score
,<int>,<dbl>,<dbl>
1,200003,3,4
2,200006,15,0
3,200007,15,0
4,200009,3,4
5,200012,15,0
6,200014,3,4


In [116]:
# Merge info 
df_icu <- df_icu %>%
  left_join(
    df_cns %>% select(icustay_id, gcs, cns_score), 
    by = "icustay_id"
  )

##  Coagulation SOFA score per ICU stay (Platelets) 

For the SOFA coagulation component, you typically need platelet count. In clinical SOFA scoring:

Platelets (x10³/µL)	SOFA Coag Score
≥150	0
100–149	1
50–99	2
20–49	3
<20	4

In [117]:
sql_coag <- "
SELECT icu.icustay_id,
       MIN(lab.valuenum) AS platelets
FROM ICUSTAYS icu
JOIN LABEVENTS lab
  ON lab.subject_id = icu.subject_id
 AND lab.hadm_id    = icu.hadm_id
WHERE lab.itemid IN (51265, 51505, 828, 51270)  -- platelet itemids
  AND lab.valuenum IS NOT NULL
  AND lab.charttime BETWEEN icu.intime 
                        AND DATE_ADD(icu.intime, INTERVAL 24 HOUR)
GROUP BY icu.icustay_id;
"

df_coag <- dbGetQuery(con, sql_coag)

df_coag <- df_coag %>%
  mutate(sofa_coag = case_when(
    platelets >= 150                          ~ 0,
    platelets >= 100 & platelets < 150        ~ 1,
    platelets >= 50  & platelets < 100        ~ 2,
    platelets >= 20  & platelets < 50         ~ 3,
    platelets < 20                             ~ 4,
    TRUE                                       ~ NA_real_
  ))


In [118]:
head(df_coag)

,icustay_id,platelets,sofa_coag
,<int>,<dbl>,<dbl>
1,200001,128,1
2,200003,109,1
3,200006,162,0
4,200007,247,0
5,200009,81,2
6,200010,227,0


In [119]:
# Merge info 
df_icu <- df_icu %>%
  left_join(
    df_coag %>% select(icustay_id, platelets, sofa_coag), 
    by = "icustay_id"
  )

## Renal (Creatinine / Urine output) SOFA score per ICU stay

Creatinine ITEMIDs:

50912 → Creatinine, mg/dL

50931 → Creatinine


In [120]:
# We need the worst creatinine in first 24 h

# MAX() → worst creatinine (higher = worse renal function)

sql_creatinine <- "
SELECT
    icu.icustay_id,
    MAX(le.valuenum) AS creatinine_mgdl
FROM ICUSTAYS icu
JOIN LABEVENTS le
  ON icu.hadm_id = le.hadm_id
WHERE le.itemid IN (50912, 50931)  -- Creatinine ITEMIDs
  AND le.charttime BETWEEN icu.intime
                        AND DATE_ADD(icu.intime, INTERVAL 24 HOUR)
  AND le.valuenum IS NOT NULL
GROUP BY icu.icustay_id
;
"

df_creatinine <- dbGetQuery(con, sql_creatinine)

In [121]:
# total urine output in first 24h
sql_urine <- "
SELECT
    icu.icustay_id,
    SUM(oe.value) AS urine_ml_24h
FROM ICUSTAYS icu
JOIN OUTPUTEVENTS oe
  ON icu.icustay_id = oe.icustay_id
WHERE oe.valueuom = 'mL'  -- ensure units are mL
  AND oe.charttime BETWEEN icu.intime AND DATE_ADD(icu.intime, INTERVAL 24 HOUR)
GROUP BY icu.icustay_id
;
"

df_urine <- dbGetQuery(con, sql_urine)

In [122]:
# We compute the renal SOFA score
df_renal <- df_creatinine %>%
  left_join(df_urine, by="icustay_id") %>%
  mutate(
    renal_score = case_when(
      urine_ml_24h < 500 ~ 4,
      creatinine_mgdl >= 5.0 ~ 4,
      creatinine_mgdl >= 3.5 ~ 3,
      creatinine_mgdl >= 2.0 ~ 2,
      creatinine_mgdl >= 1.2 ~ 1,
      creatinine_mgdl < 1.2 ~ 0,
      TRUE ~ NA_real_
    )
  )

In [123]:
head(df_renal)

,icustay_id,creatinine_mgdl,urine_ml_24h,renal_score
,<int>,<dbl>,<dbl>,<dbl>
1,200001,87,250,4
2,200003,159,3722,4
3,200006,71,2905,4
4,200007,233,1295,4
5,200009,91,5508,4
6,200010,127,2150,4


In [124]:
# Merge info 
df_icu <- df_icu %>%
  left_join(
    df_renal %>% select(icustay_id, creatinine_mgdl, urine_ml_24h, renal_score), 
    by = "icustay_id"
  )

## Combine all organs → Total SOFA

In [125]:
df_sofa <- df_resp %>%
  left_join(df_coag, by="icustay_id") %>%
  left_join(df_liver, by="icustay_id") %>%
  left_join(df_cv_full, by="icustay_id") %>%
  left_join(df_cns, by="icustay_id") %>%
  left_join(df_renal, by="icustay_id") %>%
  mutate(total_sofa = resp_score + sofa_coag + liver_score + 
                     cv_score + cns_score + renal_score)

In [126]:
# Merge info 
df_icu <- df_icu %>%
  left_join(
    df_sofa %>% select(icustay_id, total_sofa), 
    by = "icustay_id"
  )

### Evaluation on respiratory procedures 
 Here, we are combining the ICD procedure codes for each type of invasive ventilation.
 This allows us to distinguish between patients who required mechanical respiratory support
 at the end of their ICU stay (which is why we use this table specifically)
 and those who did not.

In [128]:
resp_support <- dbGetQuery(con, "
SELECT p.HADM_ID, p.ICD9_CODE, d.LONG_TITLE
FROM PROCEDURES_ICD p
JOIN D_ICD_PROCEDURES d ON p.ICD9_CODE = d.ICD9_CODE
WHERE LOWER(d.LONG_TITLE) LIKE '%ventilation%'
   OR LOWER(d.LONG_TITLE) LIKE '%intubation%'
")
table(resp_support$LONG_TITLE)


  Continuous invasive mechanical ventilation for 96 consecutive hours or more 
                                                                         6048 
Continuous invasive mechanical ventilation for less than 96 consecutive hours 
                                                                         9100 
           Continuous invasive mechanical ventilation of unspecified duration 
                                                                            6 
                                              Intubation of nasolacrimal duct 
                                                                            1 
                                          Non-invasive mechanical ventilation 
                                                                         2727 
                                        Other intubation of respiratory tract 
                                                                          759 

In [130]:
#str(resp_support)
resp_support <- resp_support %>%
    rename(hadm_id = HADM_ID)

 We also create a binary variable indicating the need for invasive respiratory support.
 Patients who required an invasive respiratory procedure are coded as 1,
 while patients with no procedure or only superficial/non-invasive interventions are coded as 0.

In [131]:
# Merge respiratory support procedures using hospital admission ID
df_icu <- df_icu %>%
  left_join(resp_support, by = "hadm_id") %>%
  
  # Create binary indicator for invasive respiratory support
  mutate(
    resp_procedure = if_else(
      is.na(LONG_TITLE) | trimws(LONG_TITLE) == "",
      0L,  # No invasive respiratory procedure
      1L   # Invasive respiratory procedure present
    )
  )

# Check distribution
table(df_icu$resp_procedure)


   0    1 
3661 1853 

In [133]:
str(df_icu)

'data.frame':	5514 obs. of  44 variables:
 $ subject_id        : int  533 877 4502 5547 5685 5771 9094 10075 11085 11538 ...
 $ hadm_id           : int  100009 166453 178493 173495 193803 127184 101257 153175 190627 198510 ...
 $ icustay_id        : int  253656 209705 227049 261963 208236 209240 288739 263099 227247 229747 ...
 $ intime            : POSIXct, format: "2162-05-17 10:18:31" "2143-07-01 09:34:22" ...
 $ outtime           : POSIXct, format: "2162-05-19 22:05:14" "2143-07-02 16:16:21" ...
 $ hosp_admit        : POSIXct, format: "2162-05-16 15:56:00" "2143-07-01 07:15:00" ...
 $ hosp_disch        : POSIXct, format: "2162-05-21 13:37:00" "2143-07-06 13:30:00" ...
 $ gender            : chr  "M" "F" "M" "M" ...
 $ ethnicity         : chr  "WHITE" "WHITE" "WHITE" "BLACK/AFRICAN AMERICAN" ...
 $ dob               : POSIXct, format: "2101-07-30" "2084-04-30" ...
 $ dod               : POSIXct, format: NA NA ...
 $ icu_los_hours     : num  59.8 30.7 25.4 34.5 24.8 ...
 $ hosp_los_h

## Vasopressors treatments 
### Vasopressors are ICU-stay–specific
We want to extracting dobutamine and norepinephrine exposure in the first 24h of ICU.

In [135]:
# Extract ICU admission times
icu_times <- dbGetQuery(con, "
  SELECT ICUSTAY_ID, INTIME
  FROM ICUSTAYS
")
#---------------------------------------------------------------
# DOBUTAMINE 
dobutamine <- dbGetQuery(con, "
  SELECT
      ICUSTAY_ID,
      LINKORDERID,
      RATE AS vaso_rate,         -- already in mcg/kg/min
      AMOUNT AS vaso_amount,
      STARTTIME,
      ENDTIME
  FROM INPUTEVENTS_MV
  WHERE ITEMID = 221653
")
#First-24h aggregation
dobutamine_24h <- dobutamine %>%
  left_join(icu_times, by = "ICUSTAY_ID") %>%
  filter(STARTTIME <= INTIME + hours(24)) %>%
  group_by(ICUSTAY_ID) %>%
  summarise(
    max_dobutamine_rate = max(vaso_rate, na.rm = TRUE),
    any_dobutamine = as.integer(any(vaso_rate > 0)),
    .groups = "drop"
  )%>%
rename(icustay_id = ICUSTAY_ID)

#---------------------------------------------------------------
# NOREPINEPHRINE 
norepinephrine <- dbGetQuery(con, "
  SELECT
      ICUSTAY_ID,
      LINKORDERID,
      CASE
        WHEN RATEUOM = 'mg/kg/min' THEN RATE * 1000.0
        ELSE RATE
      END AS vaso_rate,           -- now consistently mcg/kg/min
      AMOUNT AS vaso_amount,
      STARTTIME,
      ENDTIME
  FROM INPUTEVENTS_MV
  WHERE ITEMID = 221906
")

#First-24h aggregation
norepi_24h <- norepinephrine %>%
  left_join(icu_times, by = "ICUSTAY_ID") %>%
  filter(STARTTIME <= INTIME + hours(24)) %>%
  group_by(ICUSTAY_ID) %>%
  summarise(
    max_norepi_rate = ifelse(
      all(is.na(vaso_rate)),
      NA_real_,
      max(vaso_rate, na.rm = TRUE)
    ),
    any_norepi = as.integer(any(vaso_rate > 0, na.rm = TRUE)),
    .groups = "drop"
  )%>%
rename(icustay_id = ICUSTAY_ID)
#---------------------------------------------------------------
# Merge into df_icu (ICU-stay level)
df_icu <- df_icu %>%
  left_join(dobutamine_24h, by = "icustay_id") %>%
  left_join(norepi_24h, by = "icustay_id")
#---------------------------------------------------------------
#Replace missing values
df_icu <- df_icu %>%
  mutate(
    any_dobutamine = replace_na(any_dobutamine, 0L),
    any_norepi     = replace_na(any_norepi, 0L),
    max_dobutamine_rate = replace_na(max_dobutamine_rate, 0),
    max_norepi_rate     = replace_na(max_norepi_rate, 0)
  )

### Export data

In [136]:
write.csv(df_icu, "df_icu.csv", row.names = FALSE)
